In [1]:
#Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import syft as sy
import sys
import pdb 
import math
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torchvision.models as models
from random import shuffle
from torch.utils.data import SubsetRandomSampler, WeightedRandomSampler
import torchvision.models as models
import datetime

W1209 12:58:46.010348 19744 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.0-rc3.so'
W1209 12:58:46.027333 19744 module_wrapper.py:139] From c:\users\florian\appdata\local\programs\python\python37\lib\site-packages\tf_encrypted-0.5.9-py3.7.egg\tf_encrypted\session.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)
use_cuda = True
kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64
np.random.seed(1)
torch.manual_seed(1)
torch.set_num_threads(4)

# Creating workers

In [3]:
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning

alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker 
chalie = sy.VirtualWorker(hook, id="chalie")  # <-- NEW: define remote worker
dave = sy.VirtualWorker(hook, id="dave")  # <-- NEW: define remote worker
evelyn = sy.VirtualWorker(hook, id="evelyn")  # <-- NEW: define remote worker
a6= sy.VirtualWorker(hook, id="a6")  # <-- NEW: define remote worker
a7= sy.VirtualWorker(hook, id="a7")  # <-- NEW: define remote worker
a8= sy.VirtualWorker(hook, id="a8")  # <-- NEW: define remote worker
a9= sy.VirtualWorker(hook, id="a9")  # <-- NEW: define remote worker
a10= sy.VirtualWorker(hook, id="a10")  # <-- NEW: define remote worker
#a11= sy.VirtualWorker(hook, id="a11")  # <-- NEW: define remote worker
#a12= sy.VirtualWorker(hook, id="a12")  # <-- NEW: define remote worker
#a13= sy.VirtualWorker(hook, id="a13")  # <-- NEW: define remote worker
#a14= sy.VirtualWorker(hook, id="a14")  # <-- NEW: define remote worker
#a15= sy.VirtualWorker(hook, id="a15")  # <-- NEW: define remote worker
#a16= sy.VirtualWorker(hook, id="a16")  # <-- NEW: define remote worker
#a17= sy.VirtualWorker(hook, id="a17")  # <-- NEW: define remote worker
#a18= sy.VirtualWorker(hook, id="a18")  # <-- NEW: define remote worker
#a19= sy.VirtualWorker(hook, id="a19")  # <-- NEW: define remote worker


#fraudulin = sy.VirtualWorker(hook, id="fraudulin")
#fraudrich = sy.VirtualWorker(hook, id="fraudrich") 

compute_nodes = [alice, bob, chalie, dave, evelyn, a6, a7, a8, a9, a10]
compute_nodes_minorityClasses = [a9, a10]
frauds = []

# Helper function for dataset loader generation

In [4]:
#def generateLoadersPerClass(dataset):
#    #loaders per class
#    loaders_per_class = []
#    for class_name in dataset.classes:
#        # get the indices in the dataset that are relative to that class
#        idx = [
#            pos for pos, item in enumerate(dataset.samples)
#            if item[1] == dataset.class_to_idx[class_name]]
#        # construct the corresponding dataloader thanks to a SubsetRandomSampler
#        loaders_per_class += [torch.utils.data.DataLoader(
#            dataset, 
#            batch_size=batch_size,
#            sampler=SubsetRandomSampler(idx),
#            **kwargs)]
#    return loaders_per_class

# Loading training & test datasets

In [5]:
data_transform = transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),                     
        transforms.Normalize(                     
            mean=[0.485, 0.456, 0.406],               
            std=[0.229, 0.224, 0.225]                  
        )])


#benign data
t = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\Datensätze_ready\\Traffic\\original_5classes_1',
                             transform=data_transform)
original_loader_majorityclasses = torch.utils.data.DataLoader(t, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
temp = [] #reset target values
for d,s in t.samples:
    if s == 0:
        temp.append((d,0))
    elif s == 1:
        temp.append((d,1))
    elif s == 2:
        temp.append((d,2))
    elif s == 3:
        temp.append((d,6))
    elif s == 4:
        temp.append((d,8))
    else:
        print("never reached")
t.samples=temp

t2 = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\Datensätze_ready\\Traffic\\original_5classes_2',
                             transform=data_transform)
temp = [] #reset target values
for d,s in t2.samples:
    if s == 0:
        temp.append((d,3))
    elif s == 1:
        temp.append((d,4))
    elif s == 2:
        temp.append((d,5))
    elif s == 3:
        temp.append((d,7))
    elif s == 4:
        temp.append((d,9))
    else:
        print("never reached")
t2.samples=temp

original_loader_minorityclasses = torch.utils.data.DataLoader(t2, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)

#benign test data
testdata = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\Datensätze_ready\\Traffic\\original_test',
                             transform=data_transform)

test_loader = torch.utils.data.DataLoader(testdata, batch_size=batch_size, **kwargs)



# Load backdoor dataset

In [6]:
#malicious data
backdoored = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\Datensätze_ready\\Traffic\\backdoors_greensquare',
                             transform=data_transform)
backdoored.samples = [(d, 0) for d, s in backdoored.samples] #set each image of backdoors to 001
backdoored_loader = torch.utils.data.DataLoader(backdoored, 
                batch_size=batch_size,
                shuffle=True,
                **kwargs)
#backdoored_loaders = generateLoadersPerClass(backdoored)

#malicious test data
backdoored_test = datasets.ImageFolder(root = 
                             'C:\\Users\Florian\\Desktop\\Datensätze_ready\\Traffic\\backdoors_greensquare_test',
                             transform=data_transform)
backdoored_test.samples = [(d, 0) for d, s in backdoored_test.samples] #set each image of backdoors to 001

dataset_loader_backdoored_test = torch.utils.data.DataLoader(backdoored_test, batch_size=batch_size, **kwargs)

# Visualize some training data

In [7]:
##Let’s visualize a few training images so as to understand the data augmentations.
#
#def imshow(inp, title=None):
#    """Imshow for Tensor."""
#    inp = inp.numpy().transpose((1, 2, 0))
#    mean = np.array([0.485, 0.456, 0.406])
#    std = np.array([0.229, 0.224, 0.225])
#    inp = std * inp + mean
#    inp = np.clip(inp, 0, 1)
#    plt.imshow(inp)
#    if title is not None:
#        plt.title(title)
#    plt.pause(0.001)  # pause a bit so that plots are updated
#
#class_names = t.classes
## Get a batch of training data
#inputs, classes = next(iter(original_loader_majorityclasses))
#
## Make a grid from batch
#out = torchvision.utils.make_grid(inputs)
#
#imshow(out, title=[class_names[x] for x in classes])

# Neural Network Structure

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5) #kernel size = filter size
        self.conv1 = nn.Conv2d(16, 32, 5)
        self.pool = nn.MaxPool2d(2,stride=2)           #First Max-Pooling Layer
        self.conv2 = nn.Conv2d(32, 96, 3)
        self.conv3 = nn.Conv2d(96, 256, 3)
        self.pool = nn.MaxPool2d(2, stride=2)
        self.dropout = nn.Dropout2d(p=0.37)
        self.fc0 = nn.Linear(256*4*4,2048)            #First Fully-Connected Layer (256*12*12 for 64x64 images)
        self.dropout = nn.Dropout2d(p=0.37)
        self.fc1 = nn.Linear(2048, 1024)
        self.dropout = nn.Dropout2d(p=0.37)
        self.fc2 = nn.Linear(1024, 10)
        #cannot do batchnorm after every conf layer as described in paper, because batchnorm is not supported


    def forward(self, x):
        #import pdb; pdb.set_trace()
        x = F.relu(self.conv0(x))
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        #print(x.shape)
        x = x.view(-1, 256*4*4)
        x = self.fc0(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
#http://publications.lib.chalmers.se/records/fulltext/255863/255863.pdf

# Send dataset to clients

In [9]:
train_distributed_dataset = []
train_distributed_dataset_backdoor = []


#EACH CLIENT KNOWS EACH CLASS
#normal clients
for batch_idx, (data,target) in enumerate(original_loader_majorityclasses):
            data_append = data.send(compute_nodes[batch_idx % len(compute_nodes)], inplace = True)
            target_append = target.send(compute_nodes[batch_idx % len(compute_nodes)], inplace = True)
            train_distributed_dataset.append((data_append, target_append))
            
for batch_idx, (data,target) in enumerate(original_loader_minorityclasses):
            data_append = data.send(compute_nodes_minorityClasses[batch_idx % len(compute_nodes_minorityClasses)], inplace = True)
            target_append = target.send(compute_nodes_minorityClasses[batch_idx % len(compute_nodes_minorityClasses)], inplace = True)
            train_distributed_dataset.append((data_append, target_append))

#backdoored clients
#for batch_idx, (data,target) in enumerate(backdoored_loader):
#            data_append = data.send(frauds[batch_idx % len(frauds)], inplace = True)
#            target_append = target.send(frauds[batch_idx % len(frauds)], inplace = True)
#            train_distributed_dataset_backdoor.append((data_append, target_append))
            
#shuffle list
shuffle(train_distributed_dataset)
#shuffle(train_distributed_dataset_backdoor)
#
##get subset of data to match with the number of benign and malicious nodes
#total_data = len(train_distributed_dataset) * (len(compute_nodes) + len(frauds))/len(compute_nodes)
#fraction_of_backdoored_clients = len(frauds)/(len(compute_nodes) + len(frauds))
#train_distributed_dataset_backdoor = train_distributed_dataset_backdoor[:int(total_data*fraction_of_backdoored_clients)]
#

# Training Function

In [10]:
def train(epoch, normal_or_backdoored_dataset):
    model.train()
    totalloss = 0
    number_iterations = 0
    
    for batch_idx, (data,target) in enumerate(normal_or_backdoored_dataset):   
        number_iterations +=1
        model.send(data.location) # 0) send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.cross_entropy(output, target) # 3) calculate how much we missed
        loss.backward() # 4) figure out which weights caused us to miss
        optimizer.step() # 5) change those weights
        model.get() # 6) get model (with gradients)
            
        #if batch_idx % 300 == 0:
        loss = loss.get() # <-- NEW: get the loss back
        #print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * batch_size, len(normal_or_backdoored_dataset) * batch_size,
        #        100. * batch_idx / len(normal_or_backdoored_dataset), loss.item()))
        totalloss += loss
    print('Average training loss: {}'.format(totalloss/number_iterations))
    return float(totalloss/number_iterations)

# Test Function

In [11]:
def test(model, device, test_loader, length_of_dataset):
    model.eval()
    test_loss = 0
    correct = 0    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= length_of_dataset

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, length_of_dataset,
        100. * correct / length_of_dataset))
    
    #confusion matrix
    nb_classes = 10
    confusion_matrix = torch.zeros(nb_classes, nb_classes)
    with torch.no_grad():
        for i, (inputs, classes) in enumerate(test_loader):
            inputs = inputs.to(device)
            classes = classes.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for t, p in zip(classes.view(-1), preds.view(-1)):
                    confusion_matrix[t.long(), p.long()] += 1
    print(confusion_matrix)
    print(confusion_matrix.diag()/confusion_matrix.sum(1)) #per class accuracy
         
    return test_loss, str((100. * correct / length_of_dataset))

# Run everyting

In [12]:
#helper function for learning rate
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [13]:
model = Net().to(device)

# stopped at:
#model.load_state_dict(torch.load("exp_traffic_20191125-093840_epoch_30.pt"))
optimizer = optim.SGD(model.parameters(), lr=0.01)

#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')


#set learing rate to value
#for a in range(1,31):
#    scheduler.step()

#Write to file:
dateString = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

f= open(("exp_traffic_"+dateString+".txt"),"w+")

#EXP-setup
csv_header =  "#dataset: " + "trafficsign" + "\n"
csv_header += "#way backdoor looks like: " + "green_squares" + "\n"
csv_header += "#merge strategy: " + "sequential" + "\n"
csv_header += "#number of benign sources: " + str(len(compute_nodes)) + "\n"
csv_header += "#number of malicious sources: " + str(len(frauds)) + "\n"
csv_header += "#batch size: " + str(batch_size) + "\n"
csv_header += "#distribution of data: " + "20% classes in only 20% of the clients" + "\n"
csv_header += "#percentage of poisoned data in backdoored nodes: " + "NONE" + "\n" #str(100)
csv_header += "#order of time backdoors being inserted: " + "NONE" + "\n" #backdoors first
csv_header += "#attack model: " + "basic" + "\n" #basic
csv_header += "#starttime: " + datetime.datetime.now().strftime("%H%M%S") + "\n"
csv_header += "training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp" + "\n"
print(csv_header)
f.write(csv_header)
f.close()


#RUN training
for epoch in range(1, 101):
    csv_normal = "normal;" + str(epoch) + ";" + str(get_lr(optimizer)) + ";"
    #csv_backdoor = "backdoor;" + str(epoch) + ";" + str(get_lr(optimizer)) + ";"
    
    #train normal
    avg_training_loss = train(epoch, train_distributed_dataset)
    csv_normal += str(avg_training_loss) + ";"
    timestamp_normal = datetime.datetime.now().strftime("%H%M%S")
    
    #train backdoor
    #avg_training_backdoor_loss = train(epoch, train_distributed_dataset_backdoor)
    #csv_backdoor += str(avg_training_backdoor_loss) + ";"
    #timestamp_backdoor = datetime.datetime.now().strftime("%H%M%S")
    
    #save after each 10 iterations
    if epoch % 5 == 0:
        torch.save(model.state_dict(), ("exp_traffic_"+dateString +"_epoch_" + str(epoch) + ".pt"))
        
    #test backdoor
    #test_loss, acc = test(model, device, dataset_loader_backdoored_test, len(backdoored_test))
    #csv_backdoor += str(test_loss) + ";" + acc + ";"
    
    #test normal
    test_loss, acc = test(model, device, test_loader, len(testdata))
    csv_normal += str(test_loss) + ";" + acc + ";"

    #scheduler.step(test_loss)

    csv_normal += timestamp_normal + "\n"
    #csv_backdoor += timestamp_backdoor + "\n"
    
    #Write to file
    f= open(("exp_traffic_"+dateString+".txt"),"a+")
    #f.write(csv_backdoor)
    f.write(csv_normal)
    f.close()

    



#dataset: trafficsign
#way backdoor looks like: green_squares
#merge strategy: sequential
#number of benign sources: 10
#number of malicious sources: 0
#batch size: 64
#distribution of data: 20% classes in only 20% of the clients
#percentage of poisoned data in backdoored nodes: NONE
#order of time backdoors being inserted: NONE
#attack model: basic
#starttime: 125856
training_type;epoch_number;learn_rate;avg_training_loss;avg_test_loss;test_accuracy;timestamp

Average training loss: 2.005666971206665

Test set: Average loss: 1.5689, Accuracy: 1410/3603 (39%)

tensor([[   0.,    0.,  134.,    0.,   63.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,  121.,    0.,   79.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0., 1031.,    0.,   51.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,   98.,    0.,  432.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,   72.,    0.,  372.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,   75.,    

         7.0000e+00, 6.0000e+00, 2.0000e+00, 6.3100e+02]])
tensor([0.9137, 0.8650, 0.9871, 0.9755, 0.9730, 0.9756, 0.9250, 0.9470, 1.0000,
        0.9723])
Average training loss: 0.12927550077438354

Test set: Average loss: 0.1240, Accuracy: 3485/3603 (97%)

tensor([[1.8000e+02, 6.0000e+00, 1.0000e+00, 3.0000e+00, 4.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 3.0000e+00],
        [9.0000e+00, 1.7700e+02, 1.4000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.0000e+00, 1.0680e+03, 0.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 0.0000e+00, 4.0000e+00, 3.0000e+00],
        [1.0000e+00, 0.0000e+00, 6.0000e+00, 5.2000e+02, 3.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 4.0000e+00, 1.0000e+00, 8.0000e+00, 4.2600e+02, 2.0000e+00,
         0.0000e+00, 3.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00, 0.0000e+00, 0


tensor([[1.9200e+02, 0.0000e+00, 0.0000e+00, 4.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.4000e+01, 1.8100e+02, 5.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 3.0000e+00, 1.0660e+03, 3.0000e+00, 3.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 4.0000e+00, 1.0000e+00],
        [1.0000e+00, 0.0000e+00, 5.0000e+00, 5.2400e+02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 4.0000e+00, 2.0000e+00, 7.0000e+00, 4.2700e+02, 3.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.2000e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1700e+02, 2.0000e+00, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.000

         3.0000e+00, 3.0000e+00, 2.0000e+00, 6.4000e+02]])
tensor([0.9746, 0.9050, 0.9908, 0.9868, 0.9842, 1.0000, 0.9833, 0.9621, 1.0000,
        0.9861])
Average training loss: 0.040325820446014404

Test set: Average loss: 0.0850, Accuracy: 3525/3603 (98%)

tensor([[1.9200e+02, 0.0000e+00, 0.0000e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.8000e+01, 1.8100e+02, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.0000e+00, 2.0000e+00, 1.0720e+03, 0.0000e+00, 2.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 2.0000e+00, 1.0000e+00],
        [1.0000e+00, 0.0000e+00, 5.0000e+00, 5.2400e+02, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.0000e+00, 2.0000e+00, 2.0000e+00, 6.0000e+00, 4.2500e+02, 5.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 


tensor([[1.9200e+02, 0.0000e+00, 0.0000e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.6000e+01, 1.8100e+02, 2.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 2.0000e+00, 1.0730e+03, 0.0000e+00, 3.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 4.0000e+00, 5.2500e+02, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.0000e+00, 0.0000e+00, 3.0000e+00, 1.0000e+01, 4.2900e+02, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.2000e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1800e+02, 1.0000e+00, 1.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.000

         2.0000e+00, 3.0000e+00, 2.0000e+00, 6.4200e+02]])
tensor([0.9797, 0.9350, 0.9917, 0.9906, 0.9685, 1.0000, 0.9833, 0.9621, 1.0000,
        0.9892])
Average training loss: 0.018989596515893936

Test set: Average loss: 0.0699, Accuracy: 3543/3603 (98%)

tensor([[1.9500e+02, 0.0000e+00, 0.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.7000e+01, 1.8200e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.0000e+00, 2.0000e+00, 1.0740e+03, 0.0000e+00, 2.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00],
        [1.0000e+00, 0.0000e+00, 3.0000e+00, 5.2400e+02, 2.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.0000e+00, 0.0000e+00, 2.0000e+00, 1.0000e+00, 4.3700e+02, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 


tensor([[1.9400e+02, 0.0000e+00, 0.0000e+00, 3.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.1000e+01, 1.8700e+02, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.0000e+00, 2.0000e+00, 1.0730e+03, 0.0000e+00, 2.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 3.0000e+00, 5.2400e+02, 3.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.0000e+00, 2.0000e+00, 2.0000e+00, 1.0000e+00, 4.3400e+02, 3.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.2000e+01,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1700e+02, 1.0000e+00, 1.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.000

KeyboardInterrupt: 

In [ ]:
#model = Net().to(device)
#model.load_state_dict(torch.load("newfaces_alexnet_224x224_augmented100.pt"))
#test(model, device, test_loader)


In [ ]:
len(train_distributed_dataset)

In [ ]:
len(train_distributed_dataset_backdoor)